In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from Prompts.ReACT.prompts import zeroshot_react_agent_prompt
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
OPENAI_API_KEY = os.getenv('OPEN_AI_API')

In [5]:
with open ('Prompts/prompts_routesOP/Prompt_1.txt', 'r') as file:
    query = file.read()
scratchpad = "Thought: First, I'll search for activity-oriented attractions within the cheap budget range. Action: "

In [6]:
agent_prompt = zeroshot_react_agent_prompt
def _build_agent_prompt():
    return agent_prompt.format(query=query,scratchpad=scratchpad)

def format_step(step: str) -> str:
    return step.strip('\n').strip().replace('\n', '')

In [57]:
llm = ChatAnthropic(
    model='claude-3-5-haiku-20241022',
    temperature=1,
    max_tokens=1024,
    api_key=ANTHROPIC_API_KEY,
    stop = ['.\n']
)

In [27]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=1,
    max_tokens=1024,
    api_key=OPENAI_API_KEY,
    model_kwargs = {'stop': ['\n']}
)

In [56]:
_build_agent_prompt()

'Collect information for a query plan using interleaving \'Thought\', \'Action\', and \'Observation\' steps. Ensure you gather valid information related to transportation, dining, attractions, and accommodation. All information should be written in Notebook, which will then be input into the Planner tool. Note that the nested use of tools is prohibited. Don\'t include phrases like "Action: ", "Action 5", "Thought 1", or "Thought: " in your response. \'Thought\' can reason about the current situation, and \'Action\' can have 5 different types:\n\n(1) AccommodationSearch[Budget,Preference]:\nDescription: Find the accommodation that matches the preference.\nParameters:\nPreference: A list of preferences mentioned in the query.\nExample: AccommodationSearch[Moderate Budget,[Good Location, Good Service]] would return the moderate price hotel that has a good or excellent location, as well as a good or excellent service.\n\n(2) AttractionSearch[Budget, Preference]:\nDescription: Find the attr

In [58]:
request = format_step(llm([HumanMessage(content=_build_agent_prompt())]).content)
request

"Thought: I'll start by searching for activity-oriented attractions with a cheap budget"

In [8]:
#try llama for text completion
import transformers
import torch

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline(_build_agent_prompt())

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Collect information for a query plan using interleaving \'Thought\', \'Action\', and \'Observation\' steps. Ensure you gather valid information related to transportation, dining, attractions, and accommodation. All information should be written in Notebook, which will then be input into the Planner tool. Note that the nested use of tools is prohibited. Don\'t include phrases like "Action: ", "Action 5", "Thought 1", or "Thought: " in your response. \'Thought\' can reason about the current situation, and \'Action\' can have 5 different types:\n\n(1) AccommodationSearch[Budget,Preference]:\nDescription: Find the accommodation that matches the preference.\nParameters:\nPreference: A list of preferences mentioned in the query.\nExample: AccommodationSearch[Moderate Budget,[Good Location, Good Service]] would return the moderate price hotel that has a good or excellent location, as well as a good or excellent service.\n\n(2) AttractionSearch[Budget, Preference]:\nDescri

In [9]:
pipeline(_build_agent_prompt())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Collect information for a query plan using interleaving \'Thought\', \'Action\', and \'Observation\' steps. Ensure you gather valid information related to transportation, dining, attractions, and accommodation. All information should be written in Notebook, which will then be input into the Planner tool. Note that the nested use of tools is prohibited. Don\'t include phrases like "Action: ", "Action 5", "Thought 1", or "Thought: " in your response. \'Thought\' can reason about the current situation, and \'Action\' can have 5 different types:\n\n(1) AccommodationSearch[Budget,Preference]:\nDescription: Find the accommodation that matches the preference.\nParameters:\nPreference: A list of preferences mentioned in the query.\nExample: AccommodationSearch[Moderate Budget,[Good Location, Good Service]] would return the moderate price hotel that has a good or excellent location, as well as a good or excellent service.\n\n(2) AttractionSearch[Budget, Preference]:\nDescri

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-v0.1", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mixtral-8x7B-v0.1.
403 Client Error. (Request ID: Root=1-677ca99c-1eae83d372d6fdb65c187107;a2b04a14-3383-4ee8-8e92-d0e4aad5aa38)

Cannot access gated repo for url https://huggingface.co/mistralai/Mixtral-8x7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mixtral-8x7B-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mixtral-8x7B-v0.1 to ask for access.